In [8]:
#Initial imports

In [39]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.common_lib import DataReader, NBARawData

In [2]:
#Load dataset, remove redundant column 'Id_old'

In [31]:
data_reader = DataReader()
# Load Raw Train Data
df_train = data_reader.read_data(NBARawData.TRAIN)
# Load Test Raw Data
df_test = data_reader.read_data(NBARawData.TEST)

In [32]:
#For train dataframe, remove redundant column 'Id_old'
df_train.drop('id_old', axis=1, inplace=True)
df_train.columns = df_train.columns.str.strip()
df_train.describe

<bound method NDFrame.describe of          id  gp   min   pts  fgm  fga   fg%  3p made  3pa   3p%  ...  fta  \
0      3799  80  24.3   7.8  3.0  6.4  45.7      0.1  0.3  22.6  ...  2.9   
1      3800  75  21.8  10.5  4.2  7.9  55.1     -0.3 -1.0  34.9  ...  3.6   
2      3801  85  19.1   4.5  1.9  4.5  42.8      0.4  1.2  34.3  ...  0.6   
3      3802  63  19.1   8.2  3.5  6.7  52.5      0.3  0.8  23.7  ...  1.5   
4      3803  63  17.8   3.7  1.7  3.4  50.8      0.5  1.4  13.7  ...  0.5   
...     ...  ..   ...   ...  ...  ...   ...      ...  ...   ...  ...  ...   
7995  11794  32   9.2   1.8  0.7  1.8  40.3     -0.1 -0.2  23.1  ...  0.6   
7996  11795  54   6.0   1.8  0.7  1.4  48.7      0.1  0.1   3.1  ...  0.4   
7997  11796  85  28.2  10.7  4.0  9.0  45.1      0.2  0.6  23.6  ...  3.9   
7998  11797  39   7.7   2.5  1.0  2.3  40.1     -0.3 -0.5  13.3  ...  0.7   
7999  11798  49  19.2   4.8  1.7  5.1  32.6      0.7  2.4  41.3  ...  1.3   

       ft%  oreb  dreb  reb  ast  stl  bl

In [33]:
#For test dataframe, remove redundant column 'Id_old'
df_test.drop('id_old', axis=1, inplace=True)
df_test.columns = df_test.columns.str.strip()
df_test.describe

<bound method NDFrame.describe of         id  gp   min   pts  fgm   fga   fg%  3p made  3pa   3p%  ftm  fta  \
0        0  56   9.1   4.0  1.6   3.7  43.7      0.1  0.3   7.3  0.7  1.2   
1        1  43  19.3  10.1  3.7   8.1  46.0      0.6  1.7  35.1  1.8  2.5   
2        2  82  33.9  11.3  4.9  10.6  45.6      0.5  1.9  44.8  1.8  2.7   
3        3  86  44.7  18.8  6.8  15.9  42.9      0.5  1.8  13.5  4.5  6.3   
4        4  58  12.3   4.7  1.6   4.0  40.0      0.5  1.7  38.7  1.1  1.3   
...    ...  ..   ...   ...  ...   ...   ...      ...  ...   ...  ...  ...   
3794  3794  84  21.2   8.7  3.4   6.7  50.2      0.0  0.0  -4.5  1.7  2.5   
3795  3795  49  16.3   6.4  2.9   6.6  44.4     -0.1 -0.4  19.8  1.0  1.9   
3796  3796  53   9.9   2.1  0.8   1.8  43.1     -0.4 -0.6   1.9  0.6  1.0   
3797  3797  89  38.3  14.5  5.4  11.8  45.2      0.5  1.2  24.7  2.5  2.9   
3798  3798  55  12.0   3.0  1.1   2.5  42.3      0.1  0.6  11.7  0.9  1.1   

       ft%  oreb  dreb  reb  ast  stl  bl

In [34]:
# Split the original dataset into Data-Test set
#X_train, X_val, y_train, y_val  = data_reader.split_data(df_train)

#Isolate target column
target = df_train.pop('target_5yrs')

In [35]:
#Create test/train split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_train, target, test_size=0.2, random_state=8)

X_test = df_test()

Load classifiers

In [100]:
#Classifier imports
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.linear_model import LassoCV

In [116]:
#Select classifiers and display RMSE score:
regressors = [
    linear_model.LassoCV(alphas=None, cv=5, n_jobs=-1),
    linear_model.LinearRegression(),
    ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=5000, n_alphas=100, n_jobs=None,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0),
    SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False),
    linear_model.Ridge(alpha=.05)                
    ]
for regressor in regressors:
    model = regressor
    model.fit(X_train, y_train)  
    y_pred_b = model.predict(X_val)
    print(regressor)
    print("mean squared error: %.3f" % sqrt(mean_squared_error(y_val, y_pred_b)))
 

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=-1, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)
mean squared error: 0.355
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
mean squared error: 0.355
ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=5000, n_alphas=100, n_jobs=None,
             normalize=False, positive=False, precompute='auto', random_state=0,
             selection='cyclic', tol=0.0001, verbose=0)
mean squared error: 0.355
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
mean squared error: 0.375
Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [118]:
#Tune LassoCV
model = LassoCV(alphas=np.arange(0, 1, 0.01), cv=5, n_jobs=-1)

In [119]:
model.fit(X_train, y_train);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 330.07463258451315, tolerance: 0.0715546875
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 328.6457640673842, tolerance: 0.07135529296875029
  tol, rng, random, positive)
/opt/conda/lib

In [120]:
#Store model in /models
from joblib import dump
dump(model, '../models/LassoCV.joblib')

['../models/LassoCV.joblib']

In [122]:
#Print optimised model parameters
print('alpha: %f' % model.alpha_)

alpha: 0.000000


In [123]:
#Create predictions for train and validation
y_train_preds = model.predict(X_train)
y_val_preds = model.predict(X_val)

In [124]:
#Evaluate train predictions
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(mse(y_train, y_train_preds, squared=False))
print(mae(y_train, y_train_preds))
accuracy_score(y_train, y_train_preds.round())

0.3589527722381588
0.2618999744965639


0.83234375

In [125]:
#Train confusion matrix
confusion_matrix(y_train, y_train_preds.round())

array([[   5, 1069],
       [   4, 5322]])

In [126]:
#Validation set scores
y_val_prediction = model.predict(X_val)
print(mse(y_val, y_val_preds, squared=False))
print(mae(y_val, y_val_preds))
accuracy_score(y_val, y_val_preds.round())

0.3548456446123101
0.2601613168330067


0.838125

In [127]:
#Validation confusion matrix
confusion_matrix(y_val, y_val_prediction.round())

array([[   0,  257],
       [   2, 1341]])

In [128]:
#Output predictions
X_test = df_test
y_test_preds = model.predict(X_test)

In [129]:
y_test_preds.size

3799

In [130]:
y_test_preds = np.clip(y_test_preds, a_min=0, a_max=1)

In [131]:
output = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p for p in y_test_preds]})

In [132]:
output.to_csv("../reports/dt_submission_lasso_01.csv", index=False)